In [1]:
import json
import logging
import os
from  nltk import ngrams
import multiprocessing as mp
import matplotlib.pyplot
from math import log

In [2]:
def load_doc(filename):
    file = open(filename, 'r', encoding='utf-8-sig')
    text = file.read()
    file.close()
    return text

In [41]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [3]:
# gender pair list taken from GN-Glove
DEFAULT_MALE_NOUNS = load_doc('gender_words/male_word_file.txt').split('\n')
DEFAULT_FEMALE_NOUNS = load_doc('gender_words/female_word_file.txt').split('\n')

In [5]:
def gender_ratios_m_f(output_data_dir,file, model,maleCount, femaleCount):
    #n = 0
    #tot = 0 
    scoresP = []
    scoresPC = []
    counted_words = []
    WordScore = []
    with open(file,'r') as f:
        data = json.load(f)
    bias_record = {}
    for words in data:
        #if (data[words]['m']+data[words]['f']!=0 and data[words]['f']!=0 and data[words]['m']!=0):
        if (data[words]['m']+data[words]['f'] > MinCount):
            if words not in DEFAULT_MALE_NOUNS and words not in DEFAULT_FEMALE_NOUNS:
                #score = data[words]['m']/(data[words]['m']+data[words]['f'])
                #score = abs(data[words]['m']-data[words]['f'])/(data[words]['m']+data[words]['f'])
                score = abs(log((data[words]['m']+1)/(data[words]['f']+1)))
                scoresP.append(score)
                
                pm = data[words]['m'] / maleCount
                pf = data[words]['f'] / femaleCount
                score_c = abs(log((pm+0.0001)/(pf+0.0001)))
                #score = log((pm+0.0001)/(pf+0.0001))
                scoresPC.append(score_c)
                
                counted_words.append(words)
                WordScore.append(log((data[words]['m']+1)/(data[words]['f']+1)))

                
            #rec = {"b_score" : score}
            #data[words].update(rec)
            #bias_record[words] = json.dumps(data[words])
    #print(bias_record)
    output_file = os.path.join(output_data_dir, 'biased_words_m_f' + model)   
    with open(output_file,'w') as fp:
        json.dump(bias_record,fp, sort_keys=True) 
    print('words considered: ',len(scoresP))
    print("Bias_score_P: ", (sum(scoresP)/len(scoresP)))
    print("Bias_score_PC: ", (sum(scoresPC)/len(scoresPC)))
    return scoresP,scoresPC,counted_words,WordScore

In [6]:
# Taken from jtcrammer repo
def get_cooccurrences(file, data, window):           
       
    with open(file, 'r') as fp:
        #print(fp)
        sentences = fp.read()
        
    male_nouns = DEFAULT_MALE_NOUNS
    female_nouns = DEFAULT_FEMALE_NOUNS
    n_grams = ngrams(sentences.split(), window)
    
    for grams in n_grams:
        pos = 1
        m = 0 
        f = 0 
        for w in grams:
                pos+=1
                if w not in data:
                    data[w]= {"m":0, "f":0}
                
                if pos==int((window+1)/2):
                    if w in male_nouns:
                        m = 1
                    if w in female_nouns:
                        f = 1
                    if m > 0:
                        for t in grams:
                            if t not in data:
                                data[t]= {"m":0, "f":0}
                            data[t]['m']+=1
                    if f > 0:
                        for t in grams:
                            if t not in data:
                                data[t]= {"m":0, "f":0}
                            data[t]['f']+=1
    return data

In [7]:
def getGenderCounts(file):
    with open(file, 'r') as fp:
        sentences = fp.read()
    tokens = sentences.split()
    maleCount = len([w for w in tokens if w in DEFAULT_MALE_NOUNS])
    femaleCount = len([w for w in tokens if w in DEFAULT_FEMALE_NOUNS])
    return maleCount,femaleCount

In [8]:
output_data_dir = './Bias/'
txtDir = './generated/'
models = os.listdir(txtDir)
models

['model_2',
 'model_0.5',
 'model_5',
 'model_0',
 'model_0.1',
 'model_0.8',
 'model_1',
 'model_0.01']

In [12]:
models = ['model_0']

In [44]:
window=10
MinCount = 30

In [45]:
for m in models:
    data ={}
    print("Processing: ",m)
    print("Files:",len(os.listdir(txtDir + m)))
    maleCount = 0
    femaleCount = 0
    for file in os.listdir(txtDir + m):
        txt_path = txtDir + m + '/' + file
        data = get_cooccurrences(txt_path, data, window)
        mCount,fCount = getGenderCounts(txt_path)
        maleCount += mCount
        femaleCount += fCount
        #print(len(data))
    output_file = output_data_dir + m + '.json'
    with open(output_file,'w') as fp:
        json.dump(data,fp)
    print('male word ratio :', maleCount/(maleCount+femaleCount))

    _,_,words,Score = gender_ratios_m_f(output_data_dir,output_file, m, maleCount, femaleCount)

Processing:  model_0
Files: 10000
male word ratio : 0.632610978416405
words considered:  4100
Bias_score_P:  0.657429493035891
Bias_score_PC:  0.30776054469921565


In [46]:
max(Score), min(Score)

(2.8903717578961645, -2.3513752571634776)

In [53]:
mb = [w for w,s in zip(words,Score) if s > 1.65]
len(mb)

105

In [55]:
fb = [w for w,s in zip(words,Score) if s < -0.5]
len(fb)

102

In [56]:
save_doc(mb, 'maleBiasedWords.txt')

In [57]:
save_doc(fb, 'femaleBiasedWords.txt')